In [ ]:
from numpy import asarray
from keras.models import Sequential
from keras.layers import Conv1D
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
import numpy as np


In [ ]:
# Convolutional Layer 

data = [[0, 0, 0, 1, 1, 0, 0, 0],
        [0, 0, 0, 1, 1, 0, 0, 0],
        [0, 0, 0, 1, 1, 0, 0, 0],
        [0, 0, 0, 1, 1, 0, 0, 0],
        [0, 0, 0, 1, 1, 0, 0, 0],
        [0, 0, 0, 1, 1, 0, 0, 0],
        [0, 0, 0, 1, 1, 0, 0, 0],
        [0, 0, 0, 1, 1, 0, 0, 0]]

data = asarray(data)
data = data.reshape(1, 8, 8, 1)

# create model
model = Sequential()
model.add(Conv2D(filters=3, kernel_size=(3,3), input_shape=(8, 8, 1)))
model.add(MaxPooling2D())
model.summary()

# define a vertical line detector
# detector_Vertical = [[[[0]],[[1]],[[0]]],
#                     [[[0]],[[1]],[[0]]],
#                     [[[0]],[[1]],[[0]]]]

# detector_Horizontal = [[[[0]],[[0]],[[0]]],
#                       [[[1]],[[1]],[[1]]],
#                       [[[0]],[[0]],[[0]]]]

# detector_Diagonal = [[[[1]],[[0]],[[0]]],
#                     [[[0]],[[1]],[[0]]],
#                     [[[0]],[[0]],[[1]]]]

# weights = [asarray(detector_Vertical), asarray([0.0])]

# # store the weights in the model
# model.set_weights(weights)
# # confirm they were stored
# print(model.get_weights())

# # apply filter to input data
# yhat = model.predict(data)
# for r in range(yhat.shape[1]):
# 	# print each column in the row
# 	print([yhat[0,r,c,0] for c in range(yhat.shape[2])])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 6, 6, 3)           30        
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 3, 3, 3)          0         
 2D)                                                             
                                                                 
Total params: 30
Trainable params: 30
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# define a vertical line detector
detector_Vertical = [[[[0]],[[1]],[[0]]],
                    [[[0]],[[1]],[[0]]],
                    [[[0]],[[1]],[[0]]]]



detector_Horizontal = [[[[0]],[[0]],[[0]]],
                      [[[1]],[[1]],[[1]]],
                      [[[0]],[[0]],[[0]]]]

detector_Diagonal = [[[[1]],[[0]],[[0]]],
                    [[[0]],[[1]],[[0]]],
                    [[[0]],[[0]],[[1]]]]

weights = [asarray(detector_Vertical), asarray([0.0])]

# store the weights in the model
model.set_weights(weights)
# confirm they were stored
print(model.get_weights())

# apply filter to input data
yhat = model.predict(data)
for r in range(yhat.shape[1]):
	# print each column in the row
	print([yhat[0,r,c,0] for c in range(yhat.shape[2])])

ValueError: ignored

In [ ]:
conv_output = np.array(yhat)
conv_output 

array([[[[0.],
         [0.],
         [3.],
         [3.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [3.],
         [3.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [3.],
         [3.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [3.],
         [3.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [3.],
         [3.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [3.],
         [3.],
         [0.],
         [0.]]]], dtype=float32)

In [ ]:
conv_output = np.array([
    [0.0, 0.0, 3.0, 3.0, 0.0, 0.0],
[0.0, 0.0, 3.0, 3.0, 0.0, 0.0],
[0.0, 0.0, 3.0, 3.0, 0.0, 0.0],
[0.0, 0.0, 3.0, 3.0, 0.0, 0.0],
[0.0, 0.0, 3.0, 3.0, 0.0, 0.0],
[0.0, 0.0, 3.0, 3.0, 0.0, 0.0],
])

In [ ]:
def get_pools(img: np.array, pool_size: int, stride: int) -> np.array:
    # To store individual pools
    pools = []
    
    # Iterate over all row blocks (single block has `stride` rows)
    for i in np.arange(img.shape[0], step=stride):
        # Iterate over all column blocks (single block has `stride` columns)
        for j in np.arange(img.shape[0], step=stride):
            
            # Extract the current pool
            mat = img[i:i+pool_size, j:j+pool_size]
            
            # Make sure it's rectangular - has the shape identical to the pool size
            if mat.shape == (pool_size, pool_size):
                # Append to the list of pools
                pools.append(mat)
                
    # Return all pools as a Numpy array
    return np.array(pools)

In [ ]:
test_pools = get_pools(img=conv_output, pool_size=2, stride=2)
test_pools

array([[[0., 0.],
        [0., 0.]],

       [[3., 3.],
        [3., 3.]],

       [[0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.]],

       [[3., 3.],
        [3., 3.]],

       [[0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.]],

       [[3., 3.],
        [3., 3.]],

       [[0., 0.],
        [0., 0.]]])

In [ ]:
def max_pooling(pools: np.array) -> np.array:
    # Total number of pools
    num_pools = pools.shape[0]
    # Shape of the matrix after pooling - Square root of the number of pools
    # Cast it to int, as Numpy will return it as float
    # For example -> np.sqrt(16) = 4.0 -> int(4.0) = 4
    tgt_shape = (int(np.sqrt(num_pools)), int(np.sqrt(num_pools)))
    # To store the max values
    pooled = []
    
    # Iterate over all pools
    for pool in pools:
        # Append the max value only
        pooled.append(np.max(pool))
        
    # Reshape to target shape
    return np.array(pooled).reshape(tgt_shape)

In [ ]:
downsampled_featuremap = max_pooling(pools=test_pools)

In [ ]:
downsampled_featuremap.flatten()

array([0., 3., 0., 0., 3., 0., 0., 3., 0.])